In [23]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
from PIL import Image 
from tensorflow.keras import layers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense 
from tensorflow.keras.optimizers import Adam 
import tensorflow as tf 
import os


In [18]:
base_dir = 'flowers'
  
img_size = 224
batch = 64

In [19]:
# Create a data augmentor 
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2) 
test_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2) 
train_datagen = train_datagen.flow_from_directory(base_dir, target_size=(img_size, img_size), subset='training', batch_size=batch) 
test_datagen = test_datagen.flow_from_directory(base_dir, target_size=( img_size, img_size), subset='validation', batch_size=batch) 


Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [20]:
# # modelling starts using a CNN. 

model = Sequential() 
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same', 
				activation='relu', input_shape=(224, 224, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2))) 


model.add(Conv2D(filters=64, kernel_size=(3, 3), 
				padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 


model.add(Conv2D(filters=64, kernel_size=(3, 3), 
				padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 

model.add(Conv2D(filters=64, kernel_size=(3, 3), 
				padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 

model.add(Flatten()) 
model.add(Dense(512)) 
model.add(Activation('relu')) 
model.add(Dense(5, activation="softmax")) 


C:\Users\adwit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 64)        │           4,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       6,423,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,541,253 (24.95 MB)

 Trainable params: 6,541,253 (24.95 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
tf.keras.utils.plot_model( 
    model, 
    show_shapes = True, 
    show_dtype = True, 
    show_layer_activations = True
)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [29]:
! pip install pydot


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs=30
model.fit(train_datagen,epochs=epochs,validation_data=test_datagen)

Epoch 1/30


C:\Users\adwit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


55/55 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.3226 - loss: 1.5447 - val_accuracy: 0.5186 - val_loss: 1.1556
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 285s 5s/step - accuracy: 0.5465 - loss: 1.1321 - val_accuracy: 0.5698 - val_loss: 1.0507
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 282s 5s/step - accuracy: 0.6118 - loss: 1.0066 - val_accuracy: 0.6000 - val_loss: 1.0507
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 310s 6s/step - accuracy: 0.6303 - loss: 0.9623 - val_accuracy: 0.6477 - val_loss: 0.9292
Epoch 5/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 259s 5s/step - accuracy: 0.6388 - loss: 0.9111 - val_accuracy: 0.6477 - val_loss: 0.9188
Epoch 6/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 429s 8s/step - accuracy: 0.6676 - loss: 0.8859 - val_accuracy: 0.6512 - val_loss: 0.9181
Epoch 7/30
34/55 ━━━━━━━━━━━━━━━━━━━━ 3:21 10s/step - accuracy: 0.6762 - loss: 0.8115

In [ ]:
from tensorflow.keras.models import load_model 
model.save('Model.h5') 
   
# load model 
savedModel=load_model('Model.h5')

In [ ]:
from keras.preprocessing import image 

#Creating list for mapping 
list_ = ['Daisy','Dandelion','Rose','Sunflower', 'Tulip'] 

#Input image 
test_image = image.load_img('img1.jpg',target_size=(224,224)) 

#For show image 
plt.imshow(test_image) 
test_image = image.img_to_array(test_image) 
test_image = np.expand_dims(test_image,axis=0) 

# Result array 
result = savedModel.predict(test_image) 
print(result) 

#Mapping result array with the main name list 
i=0
for i in range(len(result[0])): 
if(result[0][i]==1): 
	print(list_[i]) 
	break
